## Importing Libraries

In [1]:
from openai import OpenAI
import config

## Setting OpenAI API Key

In [2]:
client = OpenAI(
  api_key=config.OPENAI_API_KEY,
)

## Testing 

In [10]:
response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are an experienced mobile developer, a web scraper, and an expert in image processing and generative AI. You will be provided with an image, which could be a screenshot of a document or PDF, or it could contain handwritten text. Your task is to extract all of the relevant data from the image and return it in a specified format."
    },
    {
      "role": "user",
      "content": [{"type":"text", 
                   "text":"What: Please help me extract the textual pieces of information from the image provided.\n\nWho: I am a professional Mobile application developer in Flutter.\n\nWhy: This question originates from a company that makes mobile applications. They are making an e-commerce app and they intend to add a feature in their app that allows the user to upload images and use generative AI to extract the relevant data from the image.\n\nWhich: The data you are supposed to extract from the image is 'Balance Sheet Date', 'Cash', 'Bank', 'Product List' (this could be more than one and have the following attributes): 'Product Name', 'Quantity', 'Delivery Charges', and 'Cost'. The wording of the aforementioned fields may differ so extract according to the meaning. Please ignore anything else.\n\nHow: Finally, return the data as a JSON response."
                   },
                   {
                      "type": "image_url",
                      "image_url": {
                        "url": "https://i.imgur.com/tvsgUl9.jpg"}
                   }
      ]
    }
  ],
  max_tokens=1000,
)


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here is the data extracted from the image in JSON format:\n\n```json\n{\n  "Balance Date": "31/03/1998",\n  "Cash": "$3,000",\n  "Bank": "Mars Galactic Bank",\n  "Product List": [\n    {\n      "Product Name": "Apple",\n      "Quantity": 36,\n      "Delivery Charges": "$10",\n      "Cost": "$1"\n    },\n    {\n      "Product Name": "Mouse",\n      "Quantity": 5,\n      "Delivery Charges": "$2",\n      "Cost": "$5"\n    }\n  ]\n}\n```', role='assistant', function_call=None, tool_calls=None))


## Displaying The Results

In [11]:
print(response.choices[0].message.content)

Here is the data extracted from the image in JSON format:

```json
{
  "Balance Date": "31/03/1998",
  "Cash": "$3,000",
  "Bank": "Mars Galactic Bank",
  "Product List": [
    {
      "Product Name": "Apple",
      "Quantity": 36,
      "Delivery Charges": "$10",
      "Cost": "$1"
    },
    {
      "Product Name": "Mouse",
      "Quantity": 5,
      "Delivery Charges": "$2",
      "Cost": "$5"
    }
  ]
}
```


In [25]:
response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are an experienced mobile developer, a web scraper, and an expert in image processing and generative AI. You will be provided with an image, which could be a screenshot of a document or PDF, or it could contain handwritten text. Your task is to extract all of the relevant data from the image and return it in a specified format."
    },
    {
      "role": "user",
      "content": [{"type":"text", 
                   "text":"What: Please help me extract the textual pieces of information from the image provided.\n\nWho: I am a professional Mobile application developer in Flutter.\n\nWhy: This question originates from a company that makes mobile applications. They are making an e-commerce app and they intend to add a feature in their app that allows the user to upload images and use generative AI to extract the relevant data from the image.\n\nWhich: The data you are supposed to extract from the image is 'Balance Sheet Date', 'Cash', 'Bank', 'Product List' (this could be more than one and have the following attributes): 'Product Name', 'Quantity', 'Delivery Charges', and 'Cost'. The wording of the aforementioned fields may differ so extract according to the meaning. Please ignore anything else.\n\nHow: Finally, return the data as a JSON response."
                   },
                   {
                      "type": "image_url",
                      "image_url": {
                        "url": "https://i.imgur.com/tvsgUl9.jpg"}
                   }
      ]
    }
  ],
  max_tokens=1000,
)


In [26]:
print(response.choices[0].message.content)

```json
{
  "Balance Sheet Date": "31/03/1998",
  "Cash": "$3,000",
  "Bank": "Mars Galactic Bank",
  "Product List": [
    {
      "Product Name": "Apple",
      "Quantity": 36,
      "Delivery Charges": "$10",
      "Cost": "$1"
    },
    {
      "Product Name": "Mouse",
      "Quantity": 5,
      "Delivery Charges": "$2",
      "Cost": "$5"
    }
  ]
}
```


## URL Correction

In [ ]:
import base64
import requests


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "data/bank_sheet_2.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {config.OPENAI_API_KEY}"
}

payload = {
  "model": "gpt-4-turbo",
  "messages": [
    {
      "role": "system",
      "content": "You are an experienced mobile developer, a web scraper, and an expert in image processing and generative AI. You will be provided with an image, which could be a screenshot of a document or PDF, or it could contain handwritten text. Your task is to extract all of the relevant data from the image and return it in a specified format."
    },
    {
      "role": "user",
      "content": [{"type":"text", 
                   "text":"What: Please help me extract the textual pieces of information from the image provided.\n\nWho: I am a professional Mobile application developer in Flutter.\n\nWhy: This question originates from a company that makes mobile applications. They are making an e-commerce app and they intend to add a feature in their app that allows the user to upload images and use generative AI to extract the relevant data from the image.\n\nWhich: The data you are supposed to extract from the image is 'Balance Sheet Date', 'Cash', 'Bank', 'Product List' (this could be more than one and have the following attributes): 'Product Name', 'Quantity', 'Delivery Charges', and 'Cost'. The wording of the aforementioned fields may differ so extract according to the meaning. Please ignore anything else.\n\nHow: Finally, return the data as a JSON response."
                   },
                   {
                      "type": "image_url",
                      "image_url": {
                      "url": f"data:image/jpg;base64,{base64_image}"
                      }                     
                   }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
print(response.json()['choices'][0]['message']['content'])

In [6]:
import os

def extract_file_type(file_path):
    x, file_extension = os.path.splitext(file_path)
    print(x)
    return file_extension

# Example usage:
file_path = "data/bank_sheet.png"
file_type = extract_file_type(file_path)
print("File type:", file_type)


data/bank_sheet
File type: .png


In [8]:
f"data:image/"+ extract_file_type(file_path) + f";base64"

data/bank_sheet


'data:image/.png;base64'